# Environmental Instruments Canada


In [1]:
import numpy as np
import numpy.random as rnd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

print("Libraries Loaded")

Libraries Loaded


These functions will generate the data and the inputs for the linear regression model. The first function "expcount" will take in the number of particles and count the number of decays based off of all the half life arguments added.

The second function will calulate all the inputs to the linear regression, these are all the sums/products of the decay coefficents for each progeny.

In [2]:
def expcount(n, *args):
    countlist = np.array([0]*12)
    
    for _ in range(n):
        templist=[int(x)//15 for x in np.cumsum([rnd.exponential(a) for a in args])]
        
        for j in templist:
            if j <12:
                countlist[j] = countlist[j]+1
    return countlist

def get_inputs(*args):
    inputs = [0]*12
    times = [0,15,30,45,60,75,90,105,120,135,150,165,180]
    for i in range(12):
        inputs[i] = inputs[i]+(np.exp(-args[0]*times[i])-np.exp(-args[0]*times[i+1]))
        progeny_sum = 0
        for r in range(len(args)):
            tmp=1
            for q in range(len(args)):
                if q==r:
                    continue
                tmp *= args[q]/(args[q]-args[r])
            progeny_sum = progeny_sum + tmp*(np.exp(-args[r]*times[i])-np.exp(-args[r]*times[i+1]))
        inputs[i] = inputs[i]+progeny_sum
    return inputs

Next these are just the calculations for the decay coefficents

In [3]:
LRn222 = 3.825*24*60*60 # the half-life for Rn-222 (in seconds)
LPo218 = 3.05*60        # the half-life for Po-218
DC1HL = np.array([LRn222,LPo218])
DC1Lambda = np.log(2)/DC1HL # the decay constants for this first decay chain in units of 1/s

LRn220 = 54.5 # the half-life for Rn-220 (in seconds)
LPo216 = 0.158        # the half-life for Po-216
DC2HL = np.array([LRn220,LPo216])
DC2Lambda = np.log(2)/DC2HL # the decay constants for this first decay chain in units of 1/s

array([0.0127183 , 4.38700747])

From here we can generate some data for our counts of Radon.

In [4]:
n=10000
m=1400
counts = expcount(n,1/DC1Lambda) + expcount(m,1/DC2Lambda)
counts

array([475, 352, 337, 284, 228, 194, 147, 138, 107,  82,  97,  46])

Using the inputs to the data and the counts, we can generate a linear regression for the model. We see that the coefficient for the Radon 222 is highly inaccurate. The coefficient for the Radon 220 is however a lot closer.

In [ ]:
X = np.array([get_inputs(*(DC1Lambda)), get_inputs(*(DC2Lambda))]).transpose()
fitX = X
fitY = counts

reg = LinearRegression(fit_intercept = False).fit(fitX, fitY)
reg.coef_

Running this multiple times we can see just how inaccurate the model is for Radon 222. We will have to find some more information to add to the model or we may have had a miscalculation previously.

In [17]:
Rn222 = []
Rn220 = []

for _ in range(10):
    counts = expcount(n,1/DC1Lambda) + expcount(m,1/DC2Lambda)
    reg = LinearRegression(fit_intercept = False).fit(fitX, counts)
    Rn222.append(reg.coef_[0])
    Rn220.append(reg.coef_[1])
    
print("The mean for Rn222 is {0} with standard deviation {1}".format(np.mean(Rn222), np.std(Rn222)))
print("The mean for Rn220 is {0} with standard deviation {1}".format(np.mean(Rn220), np.std(Rn220)))

The mean for Rn222 is 79263.03249460793 with standard deviation 139272.05335450583
The mean for Rn220 is 1374.7289711643107 with standard deviation 43.46013784665399
